### PART TWO

This is the second part of the project, and it is broadly classified into two parts
1. Fetching Data from Database, loading it, preprocess it for final visualizations in PowerBI
2. Fetching the latitude and longitude via google geocoding api to plot these points on PowerBI for increased accuracy and this final output is exported in an excel file, 'final_data.xlsx'. 

In [226]:
import re
import mysql.connector
import urllib.request,  urllib.parse, urllib.error
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import datetime
import requests
import random
import numpy as np
import pandas as pd
from time import sleep
import codecs
import time
import matplotlib.pyplot as plt
import json

In [119]:
#defining cusrsor and setting up connection with local database
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"}
mydb = mysql.connector.connect(host = 'localhost', user='root', password='pakistan')
cur = mydb.cursor(buffered=True)

In [120]:
cur.execute('use zomato')

In [143]:
#reading tables from sql database
cuisine_summary = pd.read_sql_query('select * from cuisine_summary', mydb, index_col = 'id')
time_summary = pd.read_sql_query('select * from time_summary', mydb, index_col = 'id')
minimum_requirement = pd.read_sql_query('select * from minimum_requirement', mydb, index_col = 'id')
restaurants = pd.read_sql_query('select * from restaurants', mydb, index_col = 'id')

In [144]:
#Preprocessing the raw text into analyzable format
def area_process(text):
    processed = text.split('Restaurants\n')
    return processed[0]
cuisine_summary['area_name']= cuisine_summary['area_name'].apply(area_process)
cuisine_summary['cuisine_count']= cuisine_summary['cuisine_count'].astype(int)
time_summary['area_name']= time_summary['area_name'].apply(area_process)
time_summary['count_of_restaurants']= time_summary['count_of_restaurants'].astype(int)
minimum_requirement['area_name']=minimum_requirement['area_name'].apply(area_process)
minimum_requirement['minimum_order_count']= minimum_requirement['minimum_order_count'].astype(int)
restaurants['area_name']=restaurants['area_name'].apply(area_process)

#Some more text processing to convert raw data into final features
def review_process(text):
    processed = text.split()
    processed = processed[0]
    processed = processed.strip('(')
    if processed.endswith('K'):
        x = processed[:-1]
        return float(x)*1000
    elif ',' in processed:
        y = processed.split(',')
        final = ''.join(y)
        return float(final)
    else: 
        return float(processed)
    
restaurants['review_count'] = restaurants['review_count'].apply(review_process)
restaurants['expected_delivery_time'] = restaurants['expected_delivery_time'].str.extract(r'(\d+)')
restaurants['expected_delivery_time'] = restaurants['expected_delivery_time'].astype(float)
restaurants['average_cost_per_person'] = restaurants['average_cost_per_person'].str.extract(r'(\d+)')
restaurants['average_cost_per_person'] = restaurants['average_cost_per_person'].astype(float)
restaurants['minimum_order_requirement'] = restaurants['minimum_order_requirement'].str.extract(r'(\d+)')
restaurants['minimum_order_requirement'] = restaurants['minimum_order_requirement'].astype(float)
restaurants['restaurant_rating'] = restaurants['restaurant_rating'].astype(float)

In [ ]:
# Fetching Geocoding from Google API

Key = 'removed'
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
params = dict()
latitude = dict()
longitude = dict()
for address in set(restaurants['area_name']):
    params['address'] = address
    params['key'] = Key
    final_url = base_url+ urllib.parse.urlencode(params)
    http = urllib.request.urlopen(final_url)
    data = http.read().decode()
    js = json.loads(data)
    lat = js['results'][0]['geometry']['location']['lat']
    long = js['results'][0]['geometry']['location']['lng']
    latitude[address] = lat
    longitude[address] = long

#Adding the features into the dataframes
cuisine_summary['latitude'] = [latitude[x] for x in cuisine_summary['area_name']]
cuisine_summary['longitude'] = [longitude[x] for x in cuisine_summary['area_name']]
time_summary['latitude'] = [latitude[x] for x in time_summary['area_name']]
time_summary['longitude'] = [longitude[x] for x in time_summary['area_name']]
minimum_requirement['latitude'] = [latitude[x] for x in minimum_requirement['area_name']]
minimum_requirement['longitude'] = [longitude[x] for x in minimum_requirement['area_name']]
restaurants['latitude'] = [latitude[x] for x in restaurants['area_name']]
restaurants['longitude'] = [longitude[x] for x in restaurants['area_name']]

In [212]:
#Dumping all the data in the excel format for the final analysis
with pd.ExcelWriter('final_data.xlsx') as final:
    cuisine_summary.to_excel(final, sheet_name = 'cuisine_summary')
    time_summary.to_excel(final, sheet_name = 'time_summary')
    minimum_requirement.to_excel(final, sheet_name = 'minimum_order_summary')
    restaurants.to_excel(final, sheet_name = 'restaurants_data' )

In [211]:
restaurants.head()

,area_name,restaurant_name,restaurant_rating,review_count,offered_cuisines,average_cost_per_person,expected_delivery_time,minimum_order_requirement,latitude,longitude
id,,,,,,,,,,
1,Dubai Investment Park 1 (DIP 1),Anjappar,3.4,11100.0,"Chettinad, Indian, Biryani, Chinese, Seafood",40.0,50.0,60.0,24.983439,55.15414
1,Dubai Investment Park 1 (DIP 1),Pizza Ibiza,3.7,1420.0,"Italian, Pizza",50.0,40.0,0.0,24.983439,55.15414
1,Dubai Investment Park 1 (DIP 1),Little Neighborhood Restaurant,3.9,82.0,"Healthy Food, Salad",80.0,25.0,0.0,24.983439,55.15414
1,Dubai Investment Park 1 (DIP 1),Damasco Restaurant,3.7,376.0,"Lebanese, Grill, Arabian, Middle Eastern",50.0,30.0,70.0,24.983439,55.15414
1,Dubai Investment Park 1 (DIP 1),Puranmal,4.0,1437.0,"North Indian, South Indian, Street Food, Desserts",45.0,40.0,15.0,24.983439,55.15414


### THE END

This is the end of the parsing and preprocessing of the data, the second part is the visualization which is accompained in the next PowerBI notebook, 'Visualizations'. Please have a look at the visualization notebook alongwith the below explanation for a complete overall picture. 

## TRENDS, INSIGHTS AND LIMITATIONS OF THE DATA
#### An Overview of the Insights of the PowerBI Visualizations

PowerBI visualization is divided into two sheets: 
1. Sheet one visualizes and analyzes the overall summary trends w.r.t demographics
2. Sheet two visualizes and analyzes restaurant-level granular data to see trends and insights

#### Observations
1. There are a total of 42 areas considered in this assignment, 18 unique cuisine types, four different types of delivery types with an average delivery time 42.95 minutes. 
2. A largest proportion of restaurants take more than 60 minutes to prepare and deliver Food.
3. Fast Food, Indian, Healthy Food along with cafe and desserts are few of the most consumed and available cuisines in Dubai with 13.1K, 12.9K, 12.0K. 11.0K and 10.1K restaurants offering these cuisines. 
4. One of the highest utilized cuisines is Indian Food, with 155 restaurants providing this cuisine in Bur Dubai, followed by 147 and 143 in Al Raffa and Oud Metha Respectively. 
5. More than 76k restaurants in Dubai require a minimum order value of 75 AED, followed by 76k restaurants requiring a minimum order value of 50 AED.
6. This project takes a subset of 4326 Restaurants for the purpose of the visualization and analysis. 
7. The average order cost/price for one person in Dubai is 40.60 AED
8. The Average minimum order requirement is 24.86 AED in Dubai. 
9. Largest Number of Zomato's restaurant partners are in Bar Dubai, and Rashidiya and least are in Jumeriah Island and Dubai Investment Park. 
10. The average order value (as per minimum order requirement) is highest in Jumeriah Golf Estate(45.10 AED) followed by JVC and lowest in Deira City Center(34.48 AED). 
11. The highest average restaurant ratings are in Al Sufouh 2 of 3.89, followed by Jumeriah Park of 3.87 and lowest ratings are in Umm Suqeim of 3.75, followed by Dubai Investment Park of 3.75. 

#### LIMITATIONS
1. Though I have tried my best to take a fair subset of the data, this project only takes a portion of the data which can not be an outstanding representation of the overall trends and insights. 

#### FUTURE CONSIDERATIONS
1. To include other features, menu items, price points, discounts and favourite items to sketch a better picture of the demographics and restaurant industry. 
2. Doing an NLP analysis of the reviews on Zomato's website and classify the positive reviews from negative to analyze and recommend areas of improvement for the restaurant industry. 